# Using GPT-4 turbo for policy area classification

In this notebook, we take data collected by Benoit et al. ([2016]()) to illustrate how to use GPT-4-turbo through the OpenAI chat completions API to classify texts.

## Setup

In [20]:
import os
from openai import OpenAI
import tiktoken

import pandas as pd
import json

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [21]:
MODEL = 'gpt-4-turbo-preview'
# note: if you do not have an OpenAI Plus subscription, use gpt-3.5-turbo instead

In [22]:
from typing import Union, List

# define a class to count tokens
class TokenCounter:
    def __init__(self, encoding_name: Union[str, None] = None, model: Union[str, None] = None):
        """
        Initialize the tokenizer with either a model or an encoding name.

        Args:
            encoding_name (Union[str, None]): The name of the encoding to use. Default is None.
            model (Union[str, None]): The model to use for encoding. Default is None.

        Raises:
            ValueError: If neither model nor encoding_name is provided.
            ValueError: If both model and encoding_name are provided.
        """
        # ensure that either model or encoding_name is provided
        if model is None and encoding_name is None:
            raise ValueError("Either `model` or `encoding_name` must be provided.")
        if model is not None and encoding_name is not None:
            raise ValueError("Only one of `model` or `encoding_name` can be provided.")
        if encoding_name:
            self.encoding = tiktoken.get_encoding(encoding_name)
        else:
            self.encoding = tiktoken.encoding_for_model(model)
    
    def count_tokens(self, input: Union[str, List[str]]) -> Union[int, List[int]]:
        """
        Count the number of tokens in the input.

        Args:
            input (Union[str, List[str]]): The input to tokenize. Can be a string or a list of strings.

        Returns:
            Union[int, List[int]]: The number of tokens in the input. If the input is a list, returns a list of token counts.
        """
        if isinstance(input, str):
            return len(self.encoding.encode(input))
        else:
            toks = self.encoding.encode_batch(input)
            return [len(t) for t in toks]

    def __call__(self, input: Union[str, List[str]]) -> Union[int, List[int]]:
        """
        Call the tokenizer on the input. This is equivalent to calling count_tokens.

        Args:
            input (Union[str, List[str]]): The input to tokenize. Can be a string or a list of strings.

        Returns:
            Union[int, List[int]]: The number of tokens in the input. If the input is a list, returns a list of token counts.
        """
        return self.count_tokens(input)

In [23]:
# instantiate the TokenCounter
token_counter = TokenCounter(model=MODEL)

### Define the instructions

In [ ]:
# adapt instructions from Benoit et al.'s original crowd coding instructions (see data/benoit_crowdsourced_2016/instructions/) 
instructions = """
Your task is to read sentences from political texts and judging whether they deal with economic or social policy.

The sentences you will be asked to interpret come from political party manifestos. Some of these sentences will deal with economic policy; some will deal with social policy; other sentences will deal with neither economic nor social policy. We tell you below about what we mean by "economic" and "social" policy.

First, you will read a short section from a party manifesto. For the focal sentence enclosed in triple quotes, indicate your best judgment about whether it mainly refers to economic policy, to social policy, or to neither.

For each focal sentence, choose one of the following categories: "economic", "social", or "neither". If the sentence refers to economic policy, select "economic"; if it refers to social policy, select "social". If the sentence does not refer to either policy area, select "neither".

## What is "economic" policy?

**"Economic" policies** deal with all aspects of the economy, including:

- Taxation
- Government spending
- Services provided by the government or other public bodies
- Pensions, unemployment and welfare benefits, and other state benefits
- Property, investment and share ownership, public or private
- Interest rates and exchange rates
- Regulation of economic activity, public or private
- Relations between employers, workers and trade unions

## What is "social" policy?

**"Social" policies** deal with aspects of social and moral life, relationships between social groups, and matters of national and social identity, including:

- Policing, crime, punishment and rehabilitation of offenders;
- Immigration, relations between social groups, discrimination and multiculturalism;
- The role of the state in regulating the social and moral behavior of individuals
"""

## Load and prepare the data 

In [7]:
fp = "../../data/benoit_crowdsourced_2016/benoit_crowdsourced_2016_policy_area.csv"
df = pd.read_csv(fp)
# keep only gold-standard examples
df = df[df.metadata__gold]
len(df)

506

In [9]:
df.columns

Index(['uid', 'text', 'label', 'metadata__gold', 'metadata__sentence_id',
       'metadata__pre_sentence', 'metadata__post_sentence'],
      dtype='object')

When distributing sentences to crowd workers for coding, Benoit et al. provided the sentence(s) preceeding and following the to-be-coded sentence.
We will replicate this approach and thus need to add the context sentences to the to-be-coded text

In [11]:
# construct input 
def construct_input(row):
    out = ""
    if isinstance(row['metadata__pre_sentence'], str):
        out += row['metadata__pre_sentence'].strip() + " "
    # wrap the to-be-coded sentence in triple quotes (as noted in the instructions)
    out += '"""'
    out += row['text'].strip()
    out += '"""'
    if isinstance(row['metadata__post_sentence'], str):
        out += " " + row['metadata__post_sentence'].strip()
    return out

df['input'] = df.apply(construct_input, axis=1).tolist()

## Classify texts

### Example

In [12]:
i = 4
df.label.values[i], df['input'].values[i]

(2,
 'A vast change separates the Britain of today from the Britain of the late 1970s. Is it really only such a short time ago that inflation rose to an annual rate of 27 per cent? """That the leader of the Transport and General Workers\' Union was widely seen as the most powerful man in the land?""" That a minority Labour Government, staggering from crisis to crisis on borrowed money, was nonetheless maintained in power by the Liberal Party in return for the paper concession of a Lib-Lab pact? And that Labour\'s much-vaunted pay pact with the unions collapsed in the industrial anarchy of the winter of discontent, n which the dead went unburied, rubbish piled up in the streets and the country was gripped by a creeping paralysis which Labour was powerless to cure?')

In [13]:
messages = [ 
    {"role": "system", "content": instructions},
    {"role": "user", "content": df['input'].values[4]}
]

response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    seed=42,
    temperature=0.0,
    # response_format={"type": "json_object"},
)

results = response.choices[0].message.content
results

'economic'

### Classify multiple texts

In [14]:
def classify_text(text):
    messages = [ 
        {"role": "system", "content": instructions},
        {"role": "user", "content": text}
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        seed=42,
        temperature=0.0,
        # response_format={"type": "json_object"},
    )

    results = response.choices[0].message.content
    return results

In [15]:
# take a sample of 25 texts from each label class
samples = df.groupby('label').sample(25, random_state=42)[["uid", "input", "label"]].reset_index(drop=True)
samples

,uid,input,label
0,10000621,"By such steadfastness, we have not only rebuil...",1
1,10008851,set up safe facilities for disposing of radioa...,1
2,20001231,In 1832 Britain took the first step with the G...,1
3,10010111,Conclusion: The Way Forward. The proposals out...,1
4,20000991,A fair electoral system will have that effect ...,1
...,...,...,...
70,10007441,but much has been done. The great majority of ...,3
71,40007181,Our national DNA database - the first in the w...,3
72,40007211,Since 1985 the average sentence for violence a...,3
73,40007951,Anyone convicted of a second serious sexual or...,3


Let's compute how much it will cost to request classifications for this sample: 

In [16]:
# tokens in inputs
n_tokens = samples.input.apply(token_counter.count_tokens).sum()
# add token count for instructions
n_tokens += token_counter(instructions) * len(samples)
# add token count for outputs (multiplied by cost factor for output vs. input)
n_tokens += len(samples) * 3

# comopute cost (see https://openai.com/pricing)
n_tokens/1000*0.01 # dollar cents

0.35448

**_Note:_** had we used all 506 texts, we'd pay approx. U.S. $ 3.15

In [17]:
# classify: apply custom classification function to all inputs
results = samples.input.apply(classify_text)

### Evaluate

Since the dataset records texts "true" labels (based on the authors expert judgments), we can compute standard [multi-class classification metrics](https://www.kaggle.com/code/nkitgupta/evaluation-metrics-for-multi-class-classification) by comparing true labels to GPT's classifications:

In [18]:
# evaluate: compute performance metrics
from sklearn.metrics import classification_report

id2label = {2: "economic", 1: "neither", 3: "social"}
print(classification_report(samples.label.map(id2label), results.values))

              precision    recall  f1-score   support

    economic       0.96      0.92      0.94        25
     neither       0.92      0.92      0.92        25
      social       0.92      0.96      0.94        25

    accuracy                           0.93        75
   macro avg       0.93      0.93      0.93        75
weighted avg       0.93      0.93      0.93        75

